# EFL Data Colloector
need to run EFL.py first

In [23]:
%run 'EFL.py'

## Training Engine
Machines are trained here.

In [554]:
%run 'EFL.py'
machine = Machine(
    IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
    FreeFermion(size=64,mass=1e-8,c=16.0),method='random')
try:
    machine.load()
except:
    print('File not found! Train new machine.')

INFO:tensorflow:Restoring parameters from ./machine/Ising(SSLatt(32,100,UUI))FF(64,0.00,16.0)random
File not found! Train new machine.


In [575]:
for i in range(5):
    machine.train(16, learning_rate=0.001, beta1=0.95, beta2=0.9)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))

(0.0011217159953664214, array([ 1.397,  0.248,  0.248,  0.248,  0.248]))
(0.00084997651421943928, array([ 1.400,  0.252,  0.252,  0.252,  0.253]))
(0.00019262139323147611, array([ 1.402,  0.256,  0.256,  0.256,  0.256]))
(0.001816991033878559, array([ 1.400,  0.255,  0.255,  0.255,  0.255]))
(0.0035795242507029707, array([ 1.397,  0.255,  0.255,  0.255,  0.255]))


I/O: save machine

In [519]:
machine.save()

INFO:tensorflow:Saving parameters to ./machine/Ising(SSLatt(32,100,UUI))FF(64,0.00,12.0)random


----

In [400]:
machine.session.run(machine.model.J.assign(np.array([0.26,0.05,0.05,0.05,0.05])))

array([ 0.260,  0.050,  0.050,  0.050,  0.050])

In [537]:
machine.session.run(machine.model.h)

11.090354888959125

## J Sequence
Given trained machine, run additional trainings to get the time sequence of J.
### Collect Jseq
Data format: `Jseq = {machine: [[J1,J2,...], ...], ...}`

In [509]:
try:
    Jseq = load('Jseq')
except FileNotFoundError:
    Jseq = {}
try:
    Js = Jseq[machine.info]
except KeyError:
    Js = []
print('len: ', len(Js))

len:  0


In [515]:
for i in range(5):
    machine.train(16, learning_rate=0.005, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Js.append(list(machine.session.run(machine.model.J)))
print('len: ', len(Js))
print('avg: ', np.mean(Js,axis=0))
print('std: ', np.std(Js,axis=0))

(0.0014557848474766948, array([ 1.060,  0.227,  0.227,  0.229,  0.231]))
(0.0012917933267157491, array([ 1.064,  0.227,  0.227,  0.228,  0.229]))
(0.00079330942092561117, array([ 1.064,  0.223,  0.223,  0.224,  0.224]))
(0.0025328398796976744, array([ 1.062,  0.216,  0.216,  0.216,  0.217]))
(0.00071917885137383591, array([ 1.058,  0.214,  0.214,  0.215,  0.216]))
len:  30
avg:  [ 1.050  0.238  0.238  0.239  0.239]
std:  [ 0.006  0.010  0.010  0.009  0.009]


In [518]:
Jseq[machine.info] = Js
save('Jseq', Jseq)

----

In [352]:
Js=[]

In [478]:
sorted([(mach, np.mean(Js,axis=0)) for mach, Js in Jseq.items()])

[('Ising(SSLatt(16,100,UUI))FF(32,0.00,8.0)random',
  array([ 0.663,  0.201,  0.201,  0.200])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,0.5)random',
  array([ 0.331,  0.002,  0.000,  0.000,  0.000])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,1.0)random',
  array([ 0.272,  0.000,  0.000,  0.000,  0.000])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,1.5)random',
  array([ 0.253,  0.099,  0.099,  0.099,  0.099])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,16.0)random',
  array([ 1.404,  0.245,  0.245,  0.245,  0.246])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,2.0)random',
  array([ 0.258,  0.157,  0.157,  0.157,  0.158])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,3.0)random',
  array([ 0.310,  0.196,  0.195,  0.196,  0.196])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,4.0)random',
  array([ 0.382,  0.207,  0.207,  0.207,  0.208])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,8.0)random',
  array([ 0.704,  0.229,  0.229,  0.230,  0.230]))]

## Entanglement Feature
Given trained machine, collect entropy predictions
### Collect EFcon
Entanglement feature over consecutive regions.
Data format: `EFcon = {machine: {'Ssys': [S1,...], 'Smdls': {[[S1,...], ...]}}, ...}`

In [26]:
try:
    EFcon = load('EFcon')
except FileNotFoundError:
    EFcon = {}
try:
    EF = EFcon[machine.info]
    Ssys = EF['Ssys']
    Smdls = EF['Smdls']
except KeyError:
    Ssys = list(machine.data_server.fetch('consecutive')[machine.model.Ssys])
    Smdls = []
    EF = {'Ssys': Ssys, 'Smdls': Smdls}
print('len: ', len(Smdls))

len:  0


In [35]:
for i in range(5):
    machine.train(16, check=16)
    Smdl = list(machine.session.run(machine.model.Smdl, 
                            machine.data_server.fetch('consecutive')))
    Smdls.append(Smdl)
print('len: ', len(Smdls))
print('error: ', np.mean([np.sqrt(np.mean(np.square(np.array(Smdl)/np.array(Ssys)-1))) for Smdl in Smdls]))

len:  16
error:  0.0800200996659


In [40]:
EFcon[machine.info] = {'Ssys': Ssys, 'Smdls': Smdls}
save('EFcon', EFcon)

### Collect EFmul
Entanglement feature over multiregions.
Data format: `EFmul = {machine: {n: [[Ssys, Smdl], ...], ...},...}`

In [74]:
try:
    EFmul = load('EFmul')
except FileNotFoundError:
    EFmul = {}
try:
    ndict = EFmul[machine.info]
except KeyError:
    ndict = {n:[] for n in range(1, machine.model.partitions//2+1)}
print('len: ', len(ndict[1]))

len:  56


In [72]:
for i in range(5):
    machine.train(16, check=16)
    for n, pairs in ndict.items():
        pairs += [[Ssys, Smdl] for Ssys, Smdl in zip(
                    *machine.session.run([
                        machine.model.Ssys, machine.model.Smdl], 
                        machine.data_server.fetch(('multiple',n))))]
print('len: ', len(ndict[1]))
print({n:float('{0:.3f}'.format(np.sqrt(np.mean([(a/b-1.)**2 for [a, b] in pairs])))) for n, pairs in ndict.items()})

len:  56
{1: 0.075, 2: 0.073, 3: 0.045, 4: 0.038, 5: 0.037, 6: 0.025, 7: 0.026, 8: 0.037}


In [73]:
EFmul[machine.info] = ndict
save('EFmul', EFmul)